In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, explode, count, avg
from pyspark.sql.types import FloatType, ArrayType, StringType
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from textblob import TextBlob
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import seaborn as sns
import re
from collections import Counter
import pandas as pd

spark = SparkSession.builder.appName("HydrationWordCloud").getOrCreate()

data = spark.sql("select * from rnd_gssi_gold_vw.vw_clinicalstudy where ActivityName like 'Meaning of Hydration'")
filtered_data = data.filter(col("QuestionDesc").isin(["BadHydration", "GoodHydration", "HydrationDefination"]))

def _sentiment(text: str) -> float:
    try:
        return TextBlob(text).sentiment.polarity
    except:
        return 0.0  # fallback if error
sentiment_udf = udf(_sentiment, FloatType())

df_sent = filtered_data.withColumn("sentiment_score", sentiment_udf(col("SelectedAnswer")))

tokenizer = Tokenizer(inputCol="SelectedAnswer", outputCol="tokens")
df_tokenized = tokenizer.transform(df_sent)

extra_stop = ["please", "thank", "also"]
remover = StopWordsRemover() \
    .setInputCol("tokens") \
    .setOutputCol("filtered") \
    .setStopWords(StopWordsRemover.loadDefaultStopWords("english") + extra_stop)

df_filtered = remover.transform(df_tokenized)

df_keywords = (
    df_filtered
    .select(
        "QuestionId",
        "QuestionDesc",
        "SelectedAnswer",
        explode("filtered").alias("Keywords"),
        "sentiment_score"
    )
    .groupBy(
        "QuestionId",
        "QuestionDesc",
        "SelectedAnswer",
        "Keywords"
    )
    .agg(
        count("*").alias("frequency"),
        avg("sentiment_score").alias("sentiment_score")
    )
)

df_keywords.createOrReplaceGlobalTempView("keyword_counts_with_sentiment")



---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-5892716120266130>, line 5
      3 from pyspark.sql.types import FloatType, ArrayType, StringType
      4 from pyspark.ml.feature import Tokenizer, StopWordsRemover
----> 5 from textblob import TextBlob
      6 import matplotlib.pyplot as plt
      7 from wordcloud import WordCloud, STOPWORDS

ModuleNotFoundError: No module named 'textblob'